In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import statsmodels
import os
import sys
from os import listdir
from os.path import isfile, join
import functools 
from matplotlib_venn import venn3
from collections import defaultdict
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 600

In [2]:
expression_file = "/data8t/mtx/scSNV/dataset_v2/GSE57872/data/quantification_stats/gene_expression_matrix/gene_expression_matrix_cpm.csv"
exp_profile = pd.read_csv(expression_file,index_col=0)
reads_file = "/data8t/mtx/scSNV/dataset_v2/GSE57872/data/quantification_stats/gene_expression_matrix/gene_expression_matrix.csv"
reads_profile = pd.read_csv(reads_file,index_col=0)

In [59]:
pvalue_cellline = pd.read_csv('cellline.csv',index_col=0)
pvalue_patients = pd.read_csv('patients.csv',index_col=0)
pvalue_all = pd.read_csv('all_gene_snv.csv',index_col=0)

gene_patients = pd.read_csv('/data8t/mtx/scSNV/dataset_v2/GSE57872/data/processed_profile/GSE57872_final_cpm_MGH.csv', index_col=0)
snv_patients = pd.read_csv('/data8t/mtx/scSNV/dataset_v2/GSE57872/data/processed_profile/GSE57872_final_snv_MGH.csv', index_col=0)

gene_cellline = pd.read_csv('/data8t/mtx/scSNV/dataset_v2/GSE57872/data/processed_profile/GSE57872_final_cpm_CSC.csv', index_col=0)
snv_cellline = pd.read_csv('/data8t/mtx/scSNV/dataset_v2/GSE57872/data/processed_profile/GSE57872_final_snv_CSC.csv', index_col=0)

label_all = pd.read_csv('/data8t/mtx/scSNV/dataset_v2/GSE57872/data/processed_profile/GSE57872_cellline_patient_label.csv', index_col=0)


In [60]:
label_all

,label,single_or_bulk,sample_title,subtype,description
run_accession,,,,,
SRR1294493,MGH26,Single cell mRNA-seq,Single cell mRNA-seq_MGH26_A02,subtype: Pro,processed data file: GBM_data_matrix.txt
SRR1294494,MGH26,Single cell mRNA-seq,Single cell mRNA-seq_MGH26_A03,subtype: Pro+Cla,processed data file: GBM_data_matrix.txt
SRR1294496,MGH26,Single cell mRNA-seq,Single cell mRNA-seq_MGH26_A05,subtype: Pro+Cla,processed data file: GBM_data_matrix.txt
SRR1294497,MGH26,Single cell mRNA-seq,Single cell mRNA-seq_MGH26_A06,NaN,Please note that this sample did not pass the ...
SRR1294498,MGH26,Single cell mRNA-seq,Single cell mRNA-seq_MGH26_A07,subtype: Pro,processed data file: GBM_data_matrix.txt
...,...,...,...,...,...
SRR1295349,CSC8,Single cell mRNA-seq,Single cell mRNA-seq_CSC8_H06,NaN,processed data file: GBM_data_matrix.txt
SRR1295350,CSC8,Single cell mRNA-seq,Single cell mRNA-seq_CSC8_H07,NaN,processed data file: GBM_data_matrix.txt
SRR1295352,CSC8,Single cell mRNA-seq,Single cell mRNA-seq_CSC8_H09,NaN,processed data file: GBM_data_matrix.txt


# Filter out all-non-zero genes